Wav2Vec2 is a pretrained model for Automatic Speech Recognition (ASR) and was released in [September 2020](https://ai.facebook.com/blog/wav2vec-20-learning-the-structure-of-speech-from-raw-audio/) by Alexei Baevski, Michael Auli, and Alex Conneau.  Soon after the superior performance of Wav2Vec2 was demonstrated on the English ASR dataset LibriSpeech, *Facebook AI* presented XLSR-Wav2Vec2 (click [here](https://arxiv.org/abs/2006.13979)). XLSR stands for *cross-lingual  speech representations* and refers to XLSR-Wav2Vec2`s ability to learn speech representations that are useful across multiple languages.

Similar to Wav2Vec2, XLSR-Wav2Vec2 learns powerful speech representations from hundreds of thousands of hours of speech in more than 50 languages of unlabeled speech. Similar, to [BERT's masked language modeling](http://jalammar.github.io/illustrated-bert/), the model learns contextualized speech representations by randomly masking feature vectors before passing them to a transformer network.



The authors show for the first time that massively pretraining an ASR model on cross-lingual unlabeled speech data, followed by language-specific fine-tuning on very little labeled data achieves state-of-the-art results. See Table 1-5 of the official [paper](https://arxiv.org/pdf/2006.13979.pdf).

XLSR-Wav2Vec2 is fine-tuned using Connectionist Temporal Classification (CTC), which is an algorithm that is used to train neural networks for sequence-to-sequence problems and mainly in Automatic Speech Recognition and handwriting recognition. 

I highly recommend reading the blog post [Sequence Modeling with CTC (2017)](https://distill.pub/2017/ctc/) very well-written blog post by Awni Hannun.

# Initialization

Before we start, let's install both `datasets` and `transformers` from master. Also, we need the `torchaudio`, `soundfile` and `librosa` package to load audio files and the `jiwer` to evaluate our fine-tuned model using the [word error rate (WER)](https://huggingface.co/metrics/wer) metric ${}^1$.

In [ ]:
%%capture
!pip install -U transformers
!pip install datasets
!pip install fsspec==2021.5.0
!pip install torch
!pip install jiwer==2.2.0

Importing All Libraries.

In [ ]:
import os
import torch
from sklearn.model_selection import train_test_split
from datasets import load_dataset, load_metric,Dataset,concatenate_datasets,set_caching_enabled, ClassLabel
import pandas as pd

import random
from IPython.display import display, HTML

import json
from transformers import Wav2Vec2CTCTokenizer,Wav2Vec2ForCTC,Wav2Vec2Processor,Trainer,TrainingArguments,Wav2Vec2FeatureExtractor

import re
set_caching_enabled(False)
!pip install torchaudio
import soundfile as sf
import torchaudio


import IPython.display as ipd

import numpy as np
from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

from tqdm import tqdm
import torch

# Set environment variables
os.environ['WANDB_DISABLED '] = 'True'

import warnings
warnings.filterwarnings('ignore')

     |████████████████████████████████| 1.9 MB 4.3 MB/s 





---

${}^1$ In the [paper](https://arxiv.org/pdf/2006.13979.pdf), the model was evaluated using the phoneme error rate (PER), but by far the most common metric in ASR is the word error rate (WER). To keep this notebook as general as possible we decided to evaluate the model using WER.

## Prepare Data, Tokenizer, Feature Extractor

ASR models transcribe speech to text, which means that we both need a feature extractor that processes the speech signal to the model's input format, *e.g.* a feature vector, and a tokenizer that processes the model's output format to text. 

In 🤗 Transformers, the XLSR-Wav2Vec2 model is thus accompanied by both a tokenizer, called [Wav2Vec2CTCTokenizer](https://huggingface.co/transformers/master/model_doc/wav2vec2.html#wav2vec2ctctokenizer), and a feature extractor, called [Wav2Vec2FeatureExtractor](https://huggingface.co/transformers/master/model_doc/wav2vec2.html#wav2vec2featureextractor).

Let's start by creating the tokenizer responsible for decoding the model's predictions.

### Create Wav2Vec2CTCTokenizer

The pretrained Wav2Vec2 checkpoint maps the speech signal to a sequence of context representations as illustrated in the figure above. A fine-tuned XLSR-Wav2Vec2 checkpoint needs to map this sequence of context representations to its corresponding transcription so that a linear layer has to be added on top of the transformer block (shown in yellow). This linear layer is used to classifies each context representation to a token class analogous how, *e.g.*, after pretraining a linear layer is added on top of BERT's embeddings for further classification - *cf.* with *"BERT"* section of this [blog post](https://huggingface.co/blog/warm-starting-encoder-decoder).

The output size of this layer corresponds to the number of tokens in the vocabulary, which does **not** depend on XLSR-Wav2Vec2's pretraining task, but only on the labeled dataset used for fine-tuning. So in the first step, we will take a look at Common Voice and define a vocabulary based on the dataset's transcriptions.

# Loading Preprocessed Data
**🚨🚨Before running this code please run🚨🚨**:
- 1) asr-mp3-to-wav-dataset.ipynb, which have joblib code to convert all the audio files from mp3 to wav using 8 parallel processs. 
- 2) wolof-audio-noise-removing.ipynb, which has used joblib and noise cancelation to remove silence and noise from the audio file and save it in wav format.

we are now ready to load the original train file and add path to Noise removed wav folder.

In [ ]:
import logging
import transformers
transformers.logging.get_verbosity = lambda: logging.NOTSET

In [ ]:
transformers.logging.get_verbosity()


0

In [ ]:
import datasets
datasets.logging.get_verbosity = lambda: logging.NOTSET

In [ ]:
import os
import cv2
import skimage.io
from tqdm.notebook import tqdm
import zipfile
import pandas as pd
import numpy as np
import shutil
!pip install pydub
from pydub import AudioSegment
from joblib import Parallel, delayed

In [ ]:
!mkdir -p /content/WOLOF_ASR_dataset

In [ ]:
!ls /content

sample_data  SampleSubmission.csv  Test.csv  Train.csv	WOLOF_ASR_dataset


In [ ]:
data = '''{
  "title": "ASR_WOLOF_wav_16000",
  "id": "kingabzpro/ASR_WOLOF_wav_16000",
  "licenses": [
    {
      "name": "CC0-1.0"
    }
  ]
}
'''
text_file = open("/content/WOLOF_ASR_dataset/dataset-metadata.json", 'w+')
n = text_file.write(data)
text_file.close()

In [ ]:
ROOT_PATH = "/content/sample_data/drive_data/MyDrive/Audio File/Audio sample/clips/clips"
OUTPUT_DIR = "/content/WOLOF_ASR_dataset/audio_wav_16000"

In [ ]:
os.mkdir(OUTPUT_DIR)

In [ ]:
def save_fn(filename):
    
    path = f"{ROOT_PATH}/{filename}"
    save_path = f"{OUTPUT_DIR}"
    if not os.path.exists(save_path):
        os.makedirs(save_path, exist_ok=True)
    
    if os.path.exists(path):
        try:
            sound = AudioSegment.from_mp3(path)
            sound = sound.set_frame_rate(16000)
            sound.export(f"{save_path}/{filename[:-4]}.wav", format="wav")
        except:
            print(path)

In [ ]:
import os
path = "/content/sample_data/drive_data/MyDrive/Audio File/Audio sample/clips/clips"
audio_files = os.listdir(path)

In [ ]:
Parallel(n_jobs=8, backend="multiprocessing")(
    delayed(save_fn)(filename) for filename in tqdm(audio_files)
)


  0%|          | 0/10363 [00:00<?, ?it/s]

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [ ]:
!zip -r "./audio_wav_16000.zip" "/tmp/WOLOF_ASR_dataset/audio_wav_16000/"

**Audio Noise Removal**



In [ ]:
import os
import cv2
import skimage.io
from tqdm.notebook import tqdm
import zipfile
import pandas as pd
import numpy as np
import shutil
from scipy.io.wavfile import write
from pydub import AudioSegment
from joblib import Parallel, delayed
import soundfile as sf

import matplotlib.pyplot as plt
from scipy.io import wavfile
import argparse
from glob import glob
from librosa.core import resample, to_mono
from pathlib import Path
import librosa
import librosa.display
from IPython.display import Audio, IFrame, display


import warnings
warnings.filterwarnings('ignore')

In [ ]:
def envelope(y, rate, threshold):
    mask = []
    y = pd.Series(y).apply(np.abs)
    y_mean = y.rolling(window=int(rate/20),min_periods=1,center=True).max()
    for mean in y_mean:
        if mean > threshold:
            mask.append(True)
        else:
            mask.append(False)
    return mask, y_mean

def downsample_mono(path, sr):
    rate, wav = wavfile.read(path)
    wav = wav.astype(np.float32, order='F')
    try:
        tmp = wav.shape[1]
        wav = to_mono(wav.T)
    except:
        pass
    wav = resample(wav, rate, sr)
    wav = wav.astype(np.int16)
    return sr, wav

In [ ]:
ROOT_PATH = "/content/WOLOF_ASR_dataset/audio_wav_16000"
OUTPUT_DIR = "/content/WOLOF_ASR_dataset/noise_remove"

In [ ]:
os.mkdir(OUTPUT_DIR)

In [ ]:
def noise_remove(filename):
       
    path = f"{ROOT_PATH}/{filename}"
    save_path = f"{OUTPUT_DIR}"
    if not os.path.exists(save_path):
        os.makedirs(save_path, exist_ok=True)
    
    if os.path.exists(path):
        try:
            rate, wav = downsample_mono(path,16000)
            mask, env = envelope(wav, rate, threshold=200)
            sf.write(f"{save_path}/{filename}", wav[mask], rate)
        except:
            print(path)

In [ ]:
rate, wav = downsample_mono("/content/WOLOF_ASR_dataset/audio_wav_16000/0039e71c4d821271b3adfc4c5d7bd27889147d45f26717ba5be12a57c3bf62af527e1e53c3b89b9af824a1e968501043ea262d488306164919abc094083d98e3.wav",16000)
mask, env = envelope(wav, rate, threshold=200)
sf.write(f"love.wav",wav[mask],rate)

In [ ]:
import os
path = "/content/WOLOF_ASR_dataset/audio_wav_16000"
audio_files = os.listdir(path)

In [ ]:
import IPython.display as ipd
import numpy as np
import random

import soundfile as sf
import torchaudio

speech_array, sampling_rate = sf.read("./love.wav")

ipd.Audio(data=speech_array, autoplay=True, rate=16000)

In [ ]:
import IPython.display as ipd
import numpy as np
import random

import soundfile as sf
import torchaudio

speech_array, sampling_rate = sf.read("/content/WOLOF_ASR_dataset/audio_wav_16000/0039e71c4d821271b3adfc4c5d7bd27889147d45f26717ba5be12a57c3bf62af527e1e53c3b89b9af824a1e968501043ea262d488306164919abc094083d98e3.wav")

ipd.Audio(data=speech_array, autoplay=True, rate=16000)

In [ ]:
Parallel(n_jobs=8, backend="multiprocessing")(
    delayed(noise_remove)(filename) for filename in tqdm(audio_files)
)

  0%|          | 0/10363 [00:00<?, ?it/s]

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [ ]:
!zip -r "./audio_wav_16000.zip" "/content/WOLOF_ASR_dataset/noise_remove/"

Streaming output truncated to the last 5000 lines.
  adding: content/WOLOF_ASR_dataset/noise_remove/cb224c0096943f3d3b09f6ec64dad201f832c4451b79c7bb3e0ea3d0aae2b93ed64ce0cefb2c702f421d30109932cb56a4b1c96b6eb3c8ff75cb6d4bbe36cb5e.wav (deflated 26%)
  adding: content/WOLOF_ASR_dataset/noise_remove/e25cba8445e6519d3b16c7bd443d1e01d227bdbff685ac7feff08198866609a011bfe5d78fe745e1407ed8df00da252f1205096a2fee3442d5dae6906cb126b3.wav (deflated 14%)
  adding: content/WOLOF_ASR_dataset/noise_remove/af926ac793fc3c5522c9591281bd48dabdd1f52952668ed0a5121f61b2f8a5cb4deb62823dbcfd10e17f2dab0316110aaf94d44e3f420a03e1c003cad0327390.wav (deflated 13%)
  adding: content/WOLOF_ASR_dataset/noise_remove/bfc0cc2ffbd9746b69fd145983a0548df9fda3bc39c3c0129d31206fa565ae15f5a6a4c36a6e79d64076d8d6e63c6d2562706b36c4d22967508adcc098e59b5e.wav (deflated 22%)
  adding: content/WOLOF_ASR_dataset/noise_remove/70fd6afa95ef285decdc5ebc1ad0bb84b405861c3d08bbcb1ffca61a40b1dec7dc24bdd0eb73e1ebe4e8bf7f0e883e1c69087340d7e9ae36

In [ ]:
from google.colab import files
files.download("./audio_wav_16000.zip")

In [ ]:
!zip -r "./audio_to_wav_16000.zip" "/content/WOLOF_ASR_dataset/audio_wav_16000/"

Streaming output truncated to the last 5000 lines.
  adding: content/WOLOF_ASR_dataset/audio_wav_16000/cb224c0096943f3d3b09f6ec64dad201f832c4451b79c7bb3e0ea3d0aae2b93ed64ce0cefb2c702f421d30109932cb56a4b1c96b6eb3c8ff75cb6d4bbe36cb5e.wav (deflated 48%)
  adding: content/WOLOF_ASR_dataset/audio_wav_16000/e25cba8445e6519d3b16c7bd443d1e01d227bdbff685ac7feff08198866609a011bfe5d78fe745e1407ed8df00da252f1205096a2fee3442d5dae6906cb126b3.wav (deflated 14%)
  adding: content/WOLOF_ASR_dataset/audio_wav_16000/af926ac793fc3c5522c9591281bd48dabdd1f52952668ed0a5121f61b2f8a5cb4deb62823dbcfd10e17f2dab0316110aaf94d44e3f420a03e1c003cad0327390.wav (deflated 27%)
  adding: content/WOLOF_ASR_dataset/audio_wav_16000/bfc0cc2ffbd9746b69fd145983a0548df9fda3bc39c3c0129d31206fa565ae15f5a6a4c36a6e79d64076d8d6e63c6d2562706b36c4d22967508adcc098e59b5e.wav (deflated 29%)
  adding: content/WOLOF_ASR_dataset/audio_wav_16000/70fd6afa95ef285decdc5ebc1ad0bb84b405861c3d08bbcb1ffca61a40b1dec7dc24bdd0eb73e1ebe4e8bf7f0e883e1c6

In [ ]:
from google.colab import files
files.download("./audio_to_wav_16000.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from google.colab import drive
drive.mount('/content/sample_data/drive_data')

Mounted at /content/sample_data/drive_data


In [ ]:
df =pd.read_csv("/content/Train.csv")
df["votes"] = df["up_votes"]-df["down_votes"]
#Add audio path and renaming columns
df["path"] = "/content/WOLOF_ASR_dataset/noise_remove/"+df["ID"]+".wav"
print(df["ID"].head())
df.rename(columns = {'transcription':'sentence'}, inplace = True)
train,test = train_test_split(df, test_size=0.005, random_state=42)
common_voice_train_1 = Dataset.from_pandas(train[3000:])
common_voice_train_2 = Dataset.from_pandas(train[0:3000])
common_voice_test = Dataset.from_pandas(test)

0    002e50c29ac2890c7cb3b3d63dcbe512bc6850de206ca8...
1    0031672b4484f963c8a07babe6f713dd559539d44140e8...
2    00362ccc6b48d3ea225e12ddf8a06a9d582cccc03b0353...
3    0042cdb4d4a015cddacc26d88faffdd714b7a27213d2b3...
4    00439f02fa8f0dade934bdb317199b71662e9396f6bb81...
Name: ID, dtype: object
0    002e50c29ac2890c7cb3b3d63dcbe512bc6850de206ca8...
1    0031672b4484f963c8a07babe6f713dd559539d44140e8...
2    00362ccc6b48d3ea225e12ddf8a06a9d582cccc03b0353...
3    0042cdb4d4a015cddacc26d88faffdd714b7a27213d2b3...
4    00439f02fa8f0dade934bdb317199b71662e9396f6bb81...
Name: ID, dtype: object


In [ ]:
sum(df.sentence.value_counts()>1)

661

In [ ]:
train.shape,test.shape

((6649, 8), (34, 8))

Removing all the unecessary columns. 

In [ ]:
common_voice_train_1 = common_voice_train_1.remove_columns([ "ID","age",  "down_votes", "gender",  "up_votes"])
common_voice_train_2 = common_voice_train_2.remove_columns([ "ID","age",  "down_votes", "gender",  "up_votes"])

common_voice_test = common_voice_test.remove_columns([ "ID","age",  "down_votes", "gender",  "up_votes"])

In [ ]:
common_voice_train_1.shape

(3649, 4)

Let's write a short function to display some random samples of the dataset and run it a couple of times to get a feeling for the transcriptions.

In [ ]:
def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

In [ ]:
show_random_elements(common_voice_train_1.remove_columns(["path"]), num_examples=20)

,sentence,votes,__index_level_0__
0,ban oto mooy jaar Thierno Ndiaye,2,2772
1,Rond Point Sacré Coeur,2,2849
2,Nu may def pour dem,2,5733
3,Police Pikine,2,4306
4,Brioche Dorée Diamalaye,3,2081
5,Prefecture Guédiawaye,2,6400
6,Pont Mariste,2,3925
7,Terminus Serigne Assane,2,5999
8,Roi Baudouin,2,1394
9,Pont Mariste,2,3134


Alright! The transcriptions look fairly clean. 

We can see that the transcriptions contain some special characters, such as `,.?!;:`. Without a language model, it is much harder to classify speech chunks to such special characters because they don't really correspond to a characteristic sound unit. *E.g.*, the letter `"s"` has a more or less clear sound, whereas the special character `"."` does not.
Also in order to understand the meaning of a speech signal, it is usually not necessary to include special characters in the transcription.

In addition, we normalize the text to only have lower case letters and append a word separator token at the end.

In [ ]:
import re
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"\“\%\‘\”\�]'

def remove_special_characters(batch):
    batch["sentence"] = re.sub(chars_to_ignore_regex, '', batch["sentence"]).lower() + " "
    return batch

In [ ]:
common_voice_train_1 = common_voice_train_1.map(remove_special_characters)
common_voice_train_2 = common_voice_train_2.map(remove_special_characters)

common_voice_test = common_voice_test.map(remove_special_characters)

In [ ]:
show_random_elements(common_voice_train_1.remove_columns(["path"]))

,sentence,votes,__index_level_0__
0,tableau tivaoune,2,3596
1,danguou,2,6473
2,pikine,2,1462
3,ban car mooy dem académia limamoulaye,2,4915
4,ban oto mooy dem gendarmerie pikine,2,2999
5,pompiers,2,4957
6,mosquée des mamelles,2,261
7,ministère de la santé et de la prévention,2,1710
8,cimetiere bétoir,2,5795
9,ban oto mooy dem rond point ngor,2,1830


Good! This looks better. We have removed most special characters from transcriptions and normalized them to lower-case only.

In CTC, it is common to classify speech chunks into letters, so we will do the same here. 
Let's extract all distinct letters of the training and test data and build our vocabulary from this set of letters.

We write a mapping function that concatenates all transcriptions into one long transcription and then transforms the string into a set of chars. 
It is important to pass the argument `batched=True` to the `map(...)` function so that the mapping function has access to all transcriptions at once.

In [ ]:
def extract_all_chars(batch):
  all_text = " ".join(batch["sentence"])
  vocab = list(set(all_text))
  return {"vocab": [vocab], "all_text": [all_text]}

In [ ]:
vocab_train_2 = common_voice_train_2.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=common_voice_train_2.column_names)

vocab_train_1 = common_voice_train_1.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=common_voice_train_1.column_names)
vocab_test = common_voice_test.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=common_voice_test.column_names)

Now, we create the union of all distinct letters in the training dataset and test dataset and convert the resulting list into an enumerated dictionary.

In [ ]:
vocab_list = list(set(vocab_train_2["vocab"][0]) |set(vocab_train_1["vocab"][0]) | set(vocab_test["vocab"][0]))

In [ ]:
vocab_dict = {v: k for k, v in enumerate(vocab_list)}
vocab_dict

{' ': 1,
 "'": 19,
 '(': 27,
 ')': 33,
 'a': 29,
 'b': 23,
 'c': 11,
 'd': 32,
 'e': 0,
 'f': 12,
 'g': 24,
 'h': 17,
 'i': 8,
 'j': 9,
 'k': 14,
 'l': 13,
 'm': 26,
 'n': 5,
 'o': 10,
 'p': 37,
 'q': 7,
 'r': 31,
 's': 3,
 't': 28,
 'u': 18,
 'v': 34,
 'w': 16,
 'x': 6,
 'y': 30,
 'z': 36,
 'â': 15,
 'ç': 4,
 'è': 22,
 'é': 2,
 'ë': 21,
 'î': 25,
 'ô': 20,
 'œ': 35,
 '’': 38}

Cool, we see that all letters of the alphabet occur in the dataset (which is not really surprising) and we also extracted the special characters `" "` and `'`. Note that we did not exclude those special characters because: 

- The model has to learn to predict when a word is finished or else the model prediction would always be a sequence of chars which would make it impossible to separate words from each other.
- From the transcriptions above it seems that words that include an apostrophe, so I decided to keep the apostrophe in the dataset. This might be a wrong assumption though.

One should always keep in mind that the data-preprocessing is a very important step before training your model. E.g., we don't want our model to differentiate between `a` and `A` just because we forgot to normalize the data. The difference between `a` and `A` does not depend on the "sound" of the letter at all, but more on grammatical rules - *e.g.* use a capitalized letter at the beginning of the sentence. So it is sensible to remove the difference between capitalized and non-capitalized letters so that the model has an easier time learning to transcribe speech. 

It is always advantageous to get help from a native speaker of the language you would like to transcribe to verify whether the assumptions you made are sensible. 

To make it clearer that `" "` has its own token class, we give it a more visible character `|`. In addition, we also add an "unknown" token so that the model can later deal with characters not encountered in Common Voice's training set. 

Finally, we also add a padding token that corresponds to CTC's "*blank token*". The "blank token" is a core component of the CTC algorithm. For more information, please take a look at the "Alignment" section [here](https://distill.pub/2017/ctc/).

In [ ]:
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]

In [ ]:
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
len(vocab_dict)

41

Cool, now our vocabulary is complete and consists of 39 tokens, which means that the linear layer that we will add on top of the pretrained XLSR-Wav2Vec2 checkpoint will have an output dimension of 39.

Let's now save the vocabulary as a json file.

In [ ]:
with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

In a final step, we use the json file to instantiate an object of the `Wav2Vec2CTCTokenizer` class.

In [ ]:
tokenizer = Wav2Vec2CTCTokenizer("./vocab.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

Next, we will create the feature extractor.

### Create XLSR-Wav2Vec2 Feature Extractor

Speech is a continuous signal and to be treated by computers, it first has to be discretized, which is usually called **sampling**. The sampling rate hereby plays an important role in that it defines how many data points of the speech signal are measured per second. Therefore, sampling with a higher sampling rate results in a better approximation of the *real* speech signal but also necessitates more values per second.

A pretrained checkpoint expects its input data to have been sampled more or less from the same distribution as the data it was trained on. The same speech signals sampled at two different rates have a very different distribution, *e.g.*, doubling the sampling rate results in data points being twice as long. Thus, 
before fine-tuning a pretrained checkpoint of an ASR model, it is crucial to verify that the sampling rate of the data that was used to pretrain the model matches the sampling rate of the dataset used to fine-tune the model.

XLSR-Wav2Vec2 was pretrained on the audio data of [Babel](https://huggingface.co/datasets/librispeech_asr), 
[Multilingual LibriSpeech (MLS)](https://ai.facebook.com/blog/a-new-open-data-set-for-multilingual-speech-research/), and [Common Voice](https://huggingface.co/datasets/common_voice). Most of those datasets were sampled at 16kHz, so that WOLOF audio is sampled at 48kHz, has to be downsampled to 16kHz for training. Therefore, we will have to downsample our fine-tuning data to 16kHz in the following. I have already downsampled and converted my audio files in mp3-wav notebook.



A XLSR-Wav2Vec2 feature extractor object requires the following parameters to be instantiated:

- `feature_size`: Speech models take a sequence of feature vectors as an input. While the length of this sequence obviously varies, the feature size should not. In the case of Wav2Vec2, the feature size is 1 because the model was trained on the raw speech signal ${}^2$.
- `sampling_rate`: The sampling rate at which the model is trained on.
- `padding_value`: For batched inference, shorter inputs need to be padded with a specific value
- `do_normalize`: Whether the input should be *zero-mean-unit-variance* normalized or not. Usually, speech models perform better when normalizing the input
- `return_attention_mask`: Whether the model should make use of an `attention_mask` for batched inference. In general, XLSR-Wav2Vec2 models should **always** make use of the `attention_mask`.

In [ ]:
feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)

Great, XLSR-Wav2Vec2's feature extraction pipeline is thereby fully defined!

To make the usage of XLSR-Wav2Vec2 as user-friendly as possible, the feature extractor and tokenizer are *wrapped* into a single `Wav2Vec2Processor` class so that one only needs a `model` and `processor` object.

In [ ]:
processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

In [ ]:
processor.save_pretrained("./ASR WOLOF Data/wav2vec2-large-xlsr-WOLOF")

Next, we can prepare the dataset.

### Preprocess Data

So far, we have not looked at the actual values of the speech signal but just kept the path to its file in the dataset. `XLSR-Wav2Vec2` expects the audio file in the format of a 1-dimensional array, so in the first step, let's load all audio files into the dataset object.

Let's first check the serialization format of the downloaded audio files by looking at the first training sample.

In [ ]:
common_voice_train_1[0]

{'__index_level_0__': 1228,
 'path': '/content/WOLOF_ASR_dataset/noise_remove/2f00b43232666642a57e969e0180a0c50d111cd409552272ae871606838d6a222ed5ecafccd7ee0436651eece29b1365043280681f2391e4f44ae2e36921d4a7.wav',
 'sentence': 'nan laay def ngir dem tally bou bess ',
 'votes': 2}

In [ ]:
def speech_file_to_array_fn(batch):
    speech_array, sampling_rate = sf.read(batch["path"])
    batch["speech"] = speech_array
    batch["sampling_rate"] = sampling_rate
    batch["target_text"] = batch["sentence"]
    return batch
    
def speech_file_to_array_fn_test(batch):
    speech_array, sampling_rate = sf.read(batch["path"])
    batch["speech"] = speech_array
    batch["sampling_rate"] = sampling_rate
    return batch

In [ ]:
common_voice_test.column_names

['sentence', 'votes', 'path', '__index_level_0__']

In [ ]:
import os
import cv2
import skimage.io
from tqdm.notebook import tqdm
import zipfile
import pandas as pd
import numpy as np
import shutil
!pip install pydub
from pydub import AudioSegment
from joblib import Parallel, delayed

In [ ]:
common_voice_train_1 = common_voice_train_1.map(speech_file_to_array_fn, remove_columns=common_voice_train_1.column_names,num_proc=4)
common_voice_train_2 = common_voice_train_2.map(speech_file_to_array_fn, remove_columns=common_voice_train_2.column_names,num_proc=4)

common_voice_test = common_voice_test.map(speech_file_to_array_fn, remove_columns=common_voice_test.column_names,num_proc=4)

This seemed to have worked! Let's listen to a couple of audio files to better understand the dataset and verify that the audio was correctly loaded. 

**Note**: *You can click the following cell a couple of times to listen to different speech samples.*

In [ ]:
import IPython.display as ipd
import numpy as np
import random

rand_int = random.randint(0, len(common_voice_train_1)-1)

ipd.Audio(data=np.asarray(common_voice_train_1[rand_int]["speech"]), autoplay=True, rate=16000)

It can be heard, that the speakers change along with their speaking rate, accent, and background environment, etc. Overall, the recordings sound acceptably clear though, which is to be expected from a crowd-sourced read speech corpus.

Let's do a final check that the data is correctly prepared, by printing the shape of the speech input, its transcription, and the corresponding sampling rate.

**Note**: *You can click the following cell a couple of times to verify multiple samples.*

In [ ]:
rand_int = random.randint(0, len(common_voice_train_1)-1)

print("Target text:", common_voice_train_1[rand_int]["target_text"])
print("Input array shape:", np.asarray(common_voice_train_1[rand_int]["speech"]).shape)
print("Sampling rate:", common_voice_train_1[rand_int]["sampling_rate"])

Target text: ban oto mooy jaar thierno ndiaye 
Input array shape: (34633,)
Sampling rate: 16000


Good! Everything looks fine - the data is a 1-dimensional array, the sampling rate always corresponds to 16kHz, and the target text is normalized.

Finally, we can process the dataset to the format expected by the model for training. We will again make use of the `map(...)` function.

First, we check that the data samples have the same sampling rate of 16kHz.
Second, we extract the `input_values` from the loaded audio file. In our case, this includes only normalization, but for other speech models, this step could correspond to extracting, *e.g.* [Log-Mel features](https://en.wikipedia.org/wiki/Mel-frequency_cepstrum). 
Third, we encode the transcriptions to label ids.

**Note**: This mapping function is a good example of how the `Wav2Vec2Processor` class should be used. In "normal" context, calling `processor(...)` is redirected to `Wav2Vec2FeatureExtractor`'s call method. When wrapping the processor into the `as_target_processor` context, however, the same method is redirected to `Wav2Vec2CTCTokenizer`'s call method.
For more information please check the [docs](https://huggingface.co/transformers/master/model_doc/wav2vec2.html#transformers.Wav2Vec2Processor.__call__).

In [ ]:
def prepare_dataset(batch):
    # check that all files have the correct sampling rate
    assert (
        len(set(batch["sampling_rate"])) == 1
    ), f"Make sure all inputs have the same sampling rate of {processor.feature_extractor.sampling_rate}."

    batch["input_values"] = processor(batch["speech"], sampling_rate=batch["sampling_rate"][0]).input_values
    
    with processor.as_target_processor():
        batch["labels"] = processor(batch["target_text"]).input_ids
    return batch

for test fucntion we have removed `processor()` part as we dont have transcription.

In [ ]:
def prepare_dataset_test(batch):
    # check that all files have the correct sampling rate
    assert (
        len(set(batch["sampling_rate"])) == 1
    ), f"Make sure all inputs have the same sampling rate of {processor.feature_extractor.sampling_rate}."

    batch["input_values"] = processor(batch["speech"], padding=True,sampling_rate=batch["sampling_rate"][0]).input_values
    
    return batch

In [ ]:
common_voice_train_1.shape,common_voice_test.shape

((3649, 3), (34, 3))

In [ ]:
common_voice_train_1 = common_voice_train_1.map(prepare_dataset, remove_columns=common_voice_train_1.column_names, batch_size=8, num_proc=4, batched=True)
common_voice_train_2 = common_voice_train_2.map(prepare_dataset, remove_columns=common_voice_train_2.column_names, batch_size=8, num_proc=4, batched=True)

In [ ]:
common_voice_train = concatenate_datasets([common_voice_train_1, common_voice_train_2])

## Training

The data is processed so that we are ready to start setting up the training pipeline. We will make use of 🤗's [Trainer](https://huggingface.co/transformers/master/main_classes/trainer.html?highlight=trainer) for which we essentially need to do the following:

- Define a data collator. In contrast to most NLP models, XLSR-Wav2Vec2 has a much larger input length than output length. *E.g.*, a sample of input length 50000 has an output length of no more than 100. Given the large input sizes, it is much more efficient to pad the training batches dynamically meaning that all training samples should only be padded to the longest sample in their batch and not the overall longest sample. Therefore, fine-tuning XLSR-Wav2Vec2 requires a special padding data collator, which we will define below

- Evaluation metric. During training, the model should be evaluated on the word error rate. We should define a `compute_metrics` function accordingly

- Load a pretrained checkpoint. We need to load a pretrained checkpoint and configure it correctly for training.

- Define the training configuration.

After having fine-tuned the model, we will correctly evaluate it on the test data and verify that it has indeed learned to correctly transcribe speech.

### Set-up Trainer

Let's start by defining the data collator. The code for the data collator was copied from [this example](https://github.com/huggingface/transformers/blob/9a06b6b11bdfc42eea08fa91d0c737d1863c99e3/examples/research_projects/wav2vec2/run_asr.py#L81).

Without going into too many details, in contrast to the common data collators, this data collator treats the `input_values` and `labels` differently and thus applies to separate padding functions on them (again making use of XLSR-Wav2Vec2's context manager). This is necessary because in speech input and output are of different modalities meaning that they should not be treated by the same padding function.
Analogous to the common data collators, the padding tokens in the labels with `-100` so that those tokens are **not** taken into account when computing the loss.

In [ ]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [ ]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

Next, the evaluation metric is defined. As mentioned earlier, the 
predominant metric in ASR is the word error rate (WER), hence we will use it in this notebook as well.

In [ ]:
wer_metric = load_metric("wer")

The model will return a sequence of logit vectors:
$\mathbf{y}_1, \ldots, \mathbf{y}_m$ with $\mathbf{y}_1 = f_{\theta}(x_1, \ldots, x_n)[0]$ and $n >> m$.

A logit vector $\mathbf{y}_1$ contains the log-odds for each word in the vocabulary we defined earlier, thus $\text{len}(\mathbf{y}_i) =$ `config.vocab_size`. We are interested in the most likely prediction of the model and thus take the `argmax(...)` of the logits. Also, we transform the encoded labels back to the original string by replacing `-100` with the `pad_token_id` and decoding the ids while making sure that consecutive tokens are **not** grouped to the same token in CTC style ${}^1$.

In [ ]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

Now, we can load the pretrained `XLSR-Wav2Vec2` checkpoint. The tokenizer's `pad_token_id` must be to define the model's `pad_token_id` or in the case of `Wav2Vec2ForCTC` also CTC's *blank token* ${}^2$. To save GPU memory, we enable PyTorch's [gradient checkpointing](https://pytorch.org/docs/stable/checkpoint.html) and also set the loss reduction to "*mean*".

Because the dataset is quite small and because WOLO audio files are quite noisy, fine-tuning Facebook's [wav2vec2-large-xlsr-53 checkpoint](https://huggingface.co/facebook/wav2vec2-large-xlsr-53) seems to require some hyper-parameter tuning. Therefore, I had to play around a bit with different values for dropout, [SpecAugment](https://arxiv.org/abs/1904.08779)'s masking dropout rate, layer dropout, and the learning rate until training seemed to be stable enough. 


In [ ]:
len(processor.tokenizer)

41

In [ ]:
model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-large-xlsr-53", 
    attention_dropout=0.1,
    hidden_dropout=0.1,
    feat_proj_dropout=0.0,
    mask_time_prob=0.05,
    layerdrop=0.1,
    gradient_checkpointing=True,
    ctc_loss_reduction="mean",
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer)
)

Some weights of the model checkpoint at facebook/wav2vec2-large-xlsr-53 were not used when initializing Wav2Vec2ForCTC: ['quantizer.weight_proj.bias', 'project_q.bias', 'quantizer.codevectors', 'quantizer.weight_proj.weight', 'project_hid.weight', 'project_hid.bias', 'project_q.weight']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-xlsr-53 and are newly initialized: ['lm_head.bias', 'lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to u

The first component of XLSR-Wav2Vec2 consists of a stack of CNN layers that are used to extract acoustically meaningful - but contextually independent - features from the raw speech signal. This part of the model has already been sufficiently trained during pretraining and as stated in the [paper](https://arxiv.org/pdf/2006.13979.pdf) does not need to be fine-tuned anymore. 
Thus, we can set the `requires_grad` to `False` for all parameters of the *feature extraction* part.

In [ ]:
model.freeze_feature_extractor()

In a final step, we define all parameters related to training. 
To give more explanation on some of the parameters:
- `group_by_length` makes training more efficient by grouping training samples of similar input length into one batch. This can significantly speed up training time by heavily reducing the overall number of useless padding tokens that are passed through the model
- `learning_rate` were heuristically tuned until fine-tuning has become stable. Note that those parameters strongly depend on the Common Voice dataset and might be suboptimal for other speech datasets.

For more explanations on other parameters, one can take a look at the [docs](https://huggingface.co/transformers/master/main_classes/trainer.html?highlight=trainer#trainingarguments).

**Note**: If one wants to save the trained models in his/her google drive the commented-out `output_dir` can be used instead.

In [ ]:
training_args = TrainingArguments(
  output_dir="./wav2vec2-large-xlsr-WOLOF",
  group_by_length=True,
  per_device_train_batch_size=16,
  gradient_accumulation_steps=2,
  evaluation_strategy="steps",
  num_train_epochs=40,
  fp16=False,
  save_steps=500,
  eval_steps=500,
  logging_steps=500,
  learning_rate=3e-4,
  warmup_steps=1000,
  save_total_limit=2,
)

Now, all instances can be passed to Trainer and we are ready to start training!

In [ ]:
trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=common_voice_train,
    eval_dataset=common_voice_test,
    tokenizer=processor.feature_extractor,
)



---

${}^1$ To allow models to become independent of the speaker rate, in CTC, consecutive tokens that are identical are simply grouped as a single token. However, the encoded labels should not be grouped when decoding since they don't correspond to the predicted tokens of the model, which is why the `group_tokens=False` parameter has to be passed. If we wouldn't pass this parameter a word like `"hello"` would incorrectly be encoded, and decoded as `"helo"`.

${}^2$ The blank token allows the model to predict a word, such as `"hello"` by forcing it to insert the blank token between the two l's. A CTC-conform prediction of `"hello"` of our model would be `[PAD] [PAD] "h" "e" "e" "l" "l" [PAD] "l" "o" "o" [PAD]`.

### Training

Training will take between 180 and 240 minutes depending on the GPU allocated to this notebook. While the trained model yields somewhat satisfying results on *Common Voice*'s test data of Turkish, it is by no means an optimally fine-tuned model. The purpose of this notebook is to demonstrate how XLSR-Wav2Vec2's [checkpoint](https://huggingface.co/facebook/wav2vec2-large-xlsr-53) can be fine-tuned on a low-resource ASR dataset.

In case you want to use this google colab to fine-tune your model, you should make sure that your training doesn't stop due to inactivity. A simple hack to prevent this is to paste the following code into the console of this tab (*right mouse click -> inspect -> Console tab and insert code*).

In [ ]:
os.environ['WANDB_MODE'] = 'dryrun'
os.environ['WANDB_DISABLED '] = 'True'

In [ ]:
common_voice_train.shape

(6649, 2)

In [ ]:
trainer.train()

***** Running training *****
  Num examples = 6649
  Num Epochs = 40
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 2
  Total optimization steps = 8320


Step,Training Loss,Validation Loss


Step,Training Loss,Validation Loss


In [ ]:
model.save_pretrained("wav2vec2-large-xlsr-WOLOF")
processor.save_pretrained("wav2vec2-large-xlsr-WOLOF")

# Submission and Inference

loading saved model

In [ ]:
model = Wav2Vec2ForCTC.from_pretrained("wav2vec2-large-xlsr-WOLOF").to("cuda")
processor = Wav2Vec2Processor.from_pretrained("wav2vec2-large-xlsr-WOLOF")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


loading our dataset, please put orginal `Test.csv` and cleaned audio file path in `val["path"]`

In [ ]:
val =pd.read_csv("../input/wolof-asr/Test.csv")
val["path"] = "../input/wolof-asr/Noise Removed/tmp/WOLOF_ASR_dataset/noise_remove/"+val["ID"]+".wav"
val.rename(columns = {'transcription':'sentence'}, inplace = True)
common_voice_val = Dataset.from_pandas(val)

removing unnecessary columns 

In [ ]:
common_voice_val = common_voice_val.remove_columns([ "ID","age",  "down_votes", "gender",  "up_votes"])

In [ ]:
val

,ID,up_votes,down_votes,age,gender,path
0,00416cff4f818d3dfd99c9178ff0e268e7575500c8baa5...,2,0,twenties,male,../input/wolof-asr/Noise Removed/tmp/WOLOF_ASR...
1,00891ba561e80e135f9d12b9fa1347f0a2560998f7ea16...,2,0,twenties,male,../input/wolof-asr/Noise Removed/tmp/WOLOF_ASR...
2,00a508027ed4edf0bd3db79f45f4ed6e1b89fba6482c10...,2,0,twenties,male,../input/wolof-asr/Noise Removed/tmp/WOLOF_ASR...
3,00ac13cd0d93e35c1ff672cc106ad94d1ea9b93fcf049a...,2,0,twenties,female,../input/wolof-asr/Noise Removed/tmp/WOLOF_ASR...
4,00c2d5baf4719bf01b990a8924e99bda043cd462147193...,2,0,twenties,male,../input/wolof-asr/Noise Removed/tmp/WOLOF_ASR...
...,...,...,...,...,...,...
1559,ff0da457e7a3986035995912803e42261c5f5f448c126b...,2,0,NaN,NaN,../input/wolof-asr/Noise Removed/tmp/WOLOF_ASR...
1560,ff1808218a15fa576c405314e4de4bda56c44f849ff1b5...,2,0,twenties,male,../input/wolof-asr/Noise Removed/tmp/WOLOF_ASR...
1561,ff5b9a45d60600e875e0a031b1d7076c9cbdeb1c48c09c...,2,0,NaN,NaN,../input/wolof-asr/Noise Removed/tmp/WOLOF_ASR...
1562,ff98e108ec61d3bd485734b83f21be77820549dab1cac1...,2,0,NaN,NaN,../input/wolof-asr/Noise Removed/tmp/WOLOF_ASR...


In [ ]:
common_voice_val.column_names

['path']

Applying speech to array fucntion especially for test data set

In [ ]:
common_voice_val = common_voice_val.map(speech_file_to_array_fn_test, remove_columns=common_voice_val.column_names)

applying prepare data set for test which is difernt from train as I have removed transcript part.

In [ ]:
common_voice_val = common_voice_val.map(prepare_dataset_test, remove_columns=common_voice_val.column_names, batch_size=8, num_proc=4, batched=True)

In [ ]:
common_voice_val.shape[0]

1564

 we can save our processed dataset and use it for faster inference.

In [ ]:
# common_voice_val.save_to_disk("/content/drive/MyDrive/ASR WOLOF Data/Pre_Data/Submission")

In [ ]:
# common_voice_val = Dataset.load_from_disk("/content/drive/MyDrive/ASR WOLOF Data/Pre_Data/Submission")

This pard is prediction part which process inputvalues and then use model to prideict logits which can later be decode.

In [ ]:
final_pred = []
for i in tqdm(range(common_voice_val.shape[0])):    
    input_dict = processor(common_voice_val[i]["input_values"], return_tensors="pt", padding=True)

    logits = model(input_dict.input_values.to("cuda")).logits

    pred_ids = torch.argmax(logits, dim=-1)[0]
    prediction = processor.decode(pred_ids)
    final_pred.append(prediction)

100%|██████████| 1564/1564 [03:53<00:00,  6.70it/s]


In [ ]:
val.columns

Index(['ID', 'up_votes', 'down_votes', 'age', 'gender', 'path'], dtype='object')

In [ ]:
val

,ID,up_votes,down_votes,age,gender,path
0,00416cff4f818d3dfd99c9178ff0e268e7575500c8baa5...,2,0,twenties,male,../input/wolof-asr/Noise Removed/tmp/WOLOF_ASR...
1,00891ba561e80e135f9d12b9fa1347f0a2560998f7ea16...,2,0,twenties,male,../input/wolof-asr/Noise Removed/tmp/WOLOF_ASR...
2,00a508027ed4edf0bd3db79f45f4ed6e1b89fba6482c10...,2,0,twenties,male,../input/wolof-asr/Noise Removed/tmp/WOLOF_ASR...
3,00ac13cd0d93e35c1ff672cc106ad94d1ea9b93fcf049a...,2,0,twenties,female,../input/wolof-asr/Noise Removed/tmp/WOLOF_ASR...
4,00c2d5baf4719bf01b990a8924e99bda043cd462147193...,2,0,twenties,male,../input/wolof-asr/Noise Removed/tmp/WOLOF_ASR...
...,...,...,...,...,...,...
1559,ff0da457e7a3986035995912803e42261c5f5f448c126b...,2,0,NaN,NaN,../input/wolof-asr/Noise Removed/tmp/WOLOF_ASR...
1560,ff1808218a15fa576c405314e4de4bda56c44f849ff1b5...,2,0,twenties,male,../input/wolof-asr/Noise Removed/tmp/WOLOF_ASR...
1561,ff5b9a45d60600e875e0a031b1d7076c9cbdeb1c48c09c...,2,0,NaN,NaN,../input/wolof-asr/Noise Removed/tmp/WOLOF_ASR...
1562,ff98e108ec61d3bd485734b83f21be77820549dab1cac1...,2,0,NaN,NaN,../input/wolof-asr/Noise Removed/tmp/WOLOF_ASR...


one values is missing to we have added dummy transcription so that I can make submission directly. Using capitalize dose improve score but not that much.

In [ ]:
val["transcription"] = final_pred
val["transcription"] = val["transcription"].str.capitalize()
val.iloc[1390,6] = "ah"

In [ ]:
val[["ID","transcription"]].to_csv("submission24.csv",index=False)

Alright! The transcription can definitely be recognized from our prediction, but it is far from being perfect. Training the model a bit longer, spending more time on the data preprocessing, and especially using a language model for decoding would certainly improve the model's overall performance. 

For a demonstration model on a low-resource language, the results are acceptable, however 🤗.

In [ ]:
val["transcription"] 

0         Aaa
1         Aaa
2         Aaa
3           A
4           A
        ...  
1559      Aaa
1560       Aa
1561       Aa
1562    Aaaaa
1563        A
Name: transcription, Length: 1564, dtype: object